# Auto Correlation Test

In [5]:
acf_test <- function(R, i, m, alpha = 0.05) {
  N <- length(R)

  # finding M
  M <- -1
  k <- 0
  while (TRUE) {
    if (i + (k + 1) * m <= N) {
      M <- k
      k <- k + 1
    } else {
      break
    }
  }
  if (M < 0) stop("No valid M")

  # finding sum rho_hat
  sumprod <- 0
  for (k in 0:M) {
    idx1 <- i + k * m
    idx2 <- i + (k + 1) * m
    sumprod <- sumprod + R[idx1] * R[idx2]
  }
  cat("\nSum of products = ", sumprod, "\n")

  rho_hat <- (1 / (M + 1)) * sumprod - 0.25

  # finding the sigma_rho_hat 
  sigma_rho_hat <- sqrt((13 * M + 7) / (12 * (M + 1)))

  # test statistic
  Z <- rho_hat / sigma_rho_hat

  #critical value
  cv <- qnorm(1 - alpha / 2)

  cat("\nH0: rho_im = 0\n")
  cat("H1: rho_im != 0\n")

  cat(sprintf("\n(1)  rho_hat =  %g\n", rho_hat))
  cat(sprintf("(2`)  sigma_rho_hat = %g\n", sigma_rho_hat))
  cat(sprintf("(3)  Z = %g\n", Z))
  cat(sprintf("(4)  Critical Value = %g\n", cv))

  cat("\nRejection Criteria abs(Z) > Critical Value\n")
  cat(sprintf("\nDecision at alpha = %g : ", alpha))
  if (abs(Z) > cv) {
    cat("REJECT H0\n")
    cat("Autocorrelation is not zero\n")
  } else {
    cat("FAIL TO REJECT H0\n")
    cat("Autocorrelation is zero\n")

  }
}

In [ ]:
acf_test(sample, i = 3, m = 5, alpha = 0.05)

# F Count Test

In [ ]:
x1 <- rnorm(20, 0, 1)
x2 <- rnorm(20, 0, 1.5)

# Plot
y <- c(x1, x2)
group <- rep(1:2, each = 20)
plot(group, y, col = group, pch = 16,
     xlab = "Group", ylab = "Values",
     main = "Count Five Test Data")

# Identify extremes and get test statistic
X <- x1 - mean(x1)
Y <- x2 - mean(x2)

extreme_X <- sum((X < min(Y)) | (X > max(Y)))
extreme_Y <- sum((Y < min(X)) | (Y > max(X)))
stat <- max(extreme_X, extreme_Y)

cat("Extreme points in X:", extreme_X, "\n")
cat("Extreme points in Y:", extreme_Y, "\n")
cat("Count Five statistic =", stat, "\n")

# Conclusion
if (stat >= 5) {
  cat("Reject H0: Variances are not equal\n")
} else {
  cat("Do NOT reject H0: Variances are equal\n")
}

# Monte Carlo estimate of significance level
count5test <- function(x, y) {
  X <- x - mean(x)
  Y <- y - mean(y)
  countX <- sum((X < min(Y)) | (X > max(Y)))
  countY <- sum((Y < min(X)) | (Y > max(X)))
  stat <- max(countX, countY)
  return(as.integer(stat >= 5))  # 1 = reject, 0 = not reject
}

# Monte Carlo simulation under equal variances
B <- 10000
rejects <- 0
for (i in 1:B) {
  x <- rnorm(20, 0, 1)
  y <- rnorm(20, 0, 1)
  rejects <- rejects + count5test(x, y)
}
sig_level <- rejects / B

cat("Estimated significance level =", sig_level, "\n")


# KS Test

In [ ]:
kstest <- function(rand) {
  sorted_rand <- sort(rand)
  n <- length(sorted_rand)
  d_plus <- numeric(n)
  d_minus <- numeric(n)
  for (i in 1:n) {
    d_plus[i] <- i / n - sorted_rand[i]
    d_minus[i] <- sorted_rand[i] - (i - 1) / n
  }
  d_plus_max <- max(d_plus)
  d_minus_max <- max(d_minus)
  d_max <- max(d_plus_max, d_minus_max)
  print(d_max)
}

# Chi- Square test for uniformity

In [ ]:
chi_square_uniformity_test <- function(rand_nums, bins = 5, alpha = 0.05) {
  N <- length(rand_nums)
  
  # Observed frequencies
  observed <- table(cut(rand_nums, breaks = seq(0, 1, length.out = bins + 1)))
  # Expected frequencies
  expected <- rep(N / bins, bins)

  # Chi-square statistic
  chi_sq_stat <- sum((observed - expected)^2 / expected)
  
  # Critical value
  cv <- qchisq(1 - alpha, df = bins - 1)
  
  # p-value
  p_val <- 1 - pchisq(chi_sq_stat, df = bins - 1)
  
  cat("\nH0: Random numbers follow Uniform(0,1)\n")
  cat("H1: Random numbers do not follow Uniform(0,1)\n")
  
  cat(sprintf("\nChi-square statistic = %g\n", chi_sq_stat))
  cat(sprintf("Critical value at alpha = %g = %g\n", alpha, cv))
  cat(sprintf("p-value = %g\n", p_val))

  cat("\nRejection Criteria: Chi-square statistic > Critical Value\n")

  cat(sprintf("\nDecision at alpha = %g : ", alpha))
  if (chi_sq_stat > cv) {
    cat("REJECT H0\n")
    cat("Conclusion: The numbers are not uniformly distributed\n")
  } else {
    cat("FAIL TO REJECT H0\n")
    cat("Conclusion: The numbers are uniformly distributed\n")
  }
}

# Chi-Square test for independence

In [ ]:
test_independence <- function(rand_nums, bins = 5, alpha = 0.05) {
  # Assign bin indices to numbers
  bin_idx <- findInterval(rand_nums, seq(0, 1, length.out = bins + 1), rightmost.closed = TRUE)
  
  # Form contingency table of successive pairs
  pairs <- data.frame(bin1 = bin_idx[-length(bin_idx)], bin2 = bin_idx[-1])
  observed <- table(pairs$bin1, pairs$bin2)
  
  # Expected counts under independence
  row_tot <- rowSums(observed)
  col_tot <- colSums(observed)
  total <- sum(observed)
  expected <- outer(row_tot, col_tot) / total
  
  chi_sq_stat <- sum((observed - expected)^2 / expected)
  
  # Critical value
  df <- (bins - 1)^2
  cv <- qchisq(1 - alpha, df)
  p_val <- 1 - pchisq(chi_sq_stat, df)
  
  
  cat("\nH0: Successive numbers are independent\n")
  cat("H1: Successive numbers are not independent\n")

  cat(sprintf("\nChi-square statistic = %g\n", chi_sq_stat))
  cat(sprintf("Critical value at alpha = %g = %g\n", alpha, cv))
  cat(sprintf("p-value = %g\n", p_val))

  cat("\nRejection Criteria Chi-square statistic > Critical Value\n")
  
  cat(sprintf("\nDecision at alpha = %g: ", alpha))
  if (chi_sq_stat > cv) {
    cat("REJECT H0\n")
    cat("Conclusion: The numbers are NOT independent.\n")
  } else {
    cat("FAIL TO REJECT H0\n")
    cat("Conclusion: The numbers appear independent.\n")
  }
}